# A notebook with the FRB/US model
This notebook is based on the the FRB/US model packages located here (https://www.federalreserve.gov/econres/us-models-package.htm). The packages provides an Eviews (https://www.eviews.com/) implementation. The Eviews modelspecification has been transformed to ModelFlow BusinessLogic specification. 

FRB/US comes in two flawors: **VAR** in which expectaions are generated by backward looking equations, and **MCE** in which expectation are based on leaded values.

This notebook implement the **MCE** flavour. The presence of leaded variable implies that the model has to be solved for all quarters in one go. The **VAR** model which has only lagged variables can be solved for each quarter in turn. 

# Import stuff

In [6]:
from modelclass import model
import modelmf
model.modelflow_auto()

# Load the model and a baseline databank

In [16]:
mfrbusmce,basedf = model.modelload('mfrbusmce.json')

# Run the baseline 

In [17]:
baseres = mfrbusmce(basedf,'2020q1','2040q4', silent=0)

Prepare model for calculate derivatives for Newton solver
Model                           : FRB/US Forward expectations
Number of endogeneus variables  : 285
Number of derivatives           : 1359
Calculating new derivatives and create new stacked Newton solver
Will start calculating:  all lags and leads
 all lags and leads solved  
Iteration  0 Sum of distances            0.001428857786
Iteration  1 Sum of distances            0.000000026529
Solved in 1 iterations
FRB/US Forward expectations solved  


# Make a number of experiments, and run them 

In [20]:
mfrbusmce.keep_solutions = {}
for shock in [0.0, 0.005 ,0.01 ,0.1]:
    altdf = baseres.copy()
    altdf=altdf.mfcalc(f'rffintay_aerr = rffintay_aerr + {shock}')       
        
    with model.timer(f'newton all periods,shock = {shock:7} '):
        altres = mfrbusmce(altdf,keep=f'Shock={shock}',silent=1)

newton all periods,shock =     0.0  took       :         0.3259994984 Seconds
Will start calculating:  all lags and leads
 all lags and leads solved  
newton all periods,shock =   0.005  took       :        29.5 Seconds
newton all periods,shock =    0.01  took       :         1.171 Seconds
Will start calculating:  all lags and leads
 all lags and leads solved  
newton all periods,shock =     0.1  took       :        28.1 Seconds


# Display results 

In [19]:
mfrbusmce.vline = [('2020q1','   Projection')]
with mfrbusmce.set_smpl('2015q1','2040q4'):
    mfrbusmce.keep_viz('RFF')

Output()